In [ ]:
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List
from humanloop import Humanloop
from dotenv import load_dotenv
import os

""" URL TO MARKDOWN """

load_dotenv()

async def url_markdown(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:", url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:", agent_name)
        
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:", mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        
        # Save content to markdown file
        filename = f"{agent_name}.md"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(web_content)
        
        print(f"Content saved to {filename}")
    
    return filename

# Example usage:
urls = [
    "https://www.perlonlabs.com/"
]

response = await url_markdown(urls)

print("response", response)

In [13]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

load_dotenv()

client = Humanloop(
    api_key=os.getenv("HUMANLOOP_API_KEY"),
)

async def create_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""
    # Instructions:
    Based on this website content, create an engaging opening line for a customer service agent, like  <example_opening_line>"Welcome to WeCreate👋! Looking to boost your brand's online presence? What's your biggest goal right now?</example_opening_line>
    IMPORTANT: You must only respond with the opening line, no other text. No speech marks either, only the text.
    # Website content:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agent_instructions(scraped_content: str) -> str:
    user_prompt = f"""
    # Instructions:
    Based on this website content, create a short sentence that begins with the name of the Company, and a short description saying what it does, like "WeCreate, a branding & digital marketing agency based Singapore, Hong Kong, and the UK."
    IMPORTANT: You must only respond with the instructions, no other text.
    # Website content:
    {scraped_content[:1000]}
    """
    
    system_prompt = "You are an expert at creating instructions for customer service agents. You are given a website content and you are tasked with creating instructions for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:",url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:",agent_name)
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:",mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        web_content = web_content[:10000]

        # Generate opening line from content
        opening_line = await create_opening_line(web_content)
        print("\n\nopening line:",opening_line)

        company_details = await create_agent_instructions(web_content)

        print("\n\ncompany details:",company_details)
        ## abstracted away lead form content, as that's determined by the user and stored in the metadata

        humanloop_prompt = client.prompts.get(
            id="pr_BiPy5GsclDI8uAzmHR8S7",
        )
        agent_instructions = humanloop_prompt.template[0]['content'].replace("{{company_details}}", company_details)

        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        features = {
                    "lead_gen": {
                        "enabled": True
                    },
                    "callTransfer": {
                        "enabled": False
                    },
                    "notifyOnInterest": {
                        "enabled": False
                    },
                    "appointmentBooking": {
                        "enabled": False
                    }
                }
        
        form_fields = {
                "fields": [
                    {
                    "type": "text",
                    "label": "Full name"
                    },
                    {
                    "type": "email",
                    "label": "Email address"
                    }
                ]
            }
        
        """ POST to create new agent """
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": "text-chatbot-agent",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": agent_instructions,
            "features": features,
            "form_fields": form_fields,
            "company_name": agent_name
        }

        # POST request
        response = requests.post(
            "http://localhost:8000/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code == 200:
            print(f"Successfully created agent for {url}")
            return response.json()
        else:
            print(f"Failed to create agent for {url}: {response.text}")

# Example usage:
urls = [
    "https://nogood.io/"
]

response = await create_agents_from_urls(urls)
agent_id = response['data'][0]['id']

print("agent_id", agent_id)

url: https://nogood.io/
agent_name: nogood.io
hello, map_url


mapped_url: ['https://nogood.io', 'https://nogood.io/case-study-category/b2b', 'https://nogood.io/case-study-category/consumer', 'https://nogood.io/case-study-category/consumer-app', 'https://nogood.io/case-study-category/healthcare', 'https://nogood.io/case-study-category/saas', 'https://nogood.io/category/artificial-intelligence', 'https://nogood.io/category/b2b', 'https://nogood.io/category/consumer', 'https://nogood.io/category/content-marketing', 'https://nogood.io/category/cro', 'https://nogood.io/category/fintech', 'https://nogood.io/category/fractional-cmo', 'https://nogood.io/category/growth', 'https://nogood.io/category/healthcare', 'https://nogood.io/category/lifecycle-marketing', 'https://nogood.io/category/marketing-analytics', 'https://nogood.io/category/paid-search', 'https://nogood.io/category/paid-social', 'https://nogood.io/category/performance-branding', 'https://nogood.io/category/saas', 'https://nogood.

[2025-01-07 03:11:09,997][ERROR] Error processing site https://nogood.io/industry/b2b: RetryError[<Future at 0x11ac3da90 state=finished raised HTTPError>]
[2025-01-07 03:11:10,009][ERROR] Error processing site https://nogood.io/author/isabella-herrera: RetryError[<Future at 0x11a123680 state=finished raised HTTPError>]


hello, get_embedding
hello, get_embedding
KeyError occurred for site https://nogood.io/case-study/steelseries-best-use-of-data-driven-media: Missing description
hello, sliding_window_chunking


[2025-01-07 03:11:10,036][ERROR] Error processing site https://nogood.io/author/james-giller: RetryError[<Future at 0x11a1d4b90 state=finished raised HTTPError>]
[2025-01-07 03:11:10,244][ERROR] Error processing site https://nogood.io/industry/crypto: RetryError[<Future at 0x11a1d6630 state=finished raised HTTPError>]
[2025-01-07 03:11:10,246][ERROR] Error processing site https://nogood.io/case-study/nogood-steer-ev: RetryError[<Future at 0x11a945160 state=finished raised HTTPError>]
[2025-01-07 03:11:10,246][ERROR] Error processing site https://nogood.io/author/naree-asherian: RetryError[<Future at 0x11a1ea1b0 state=finished raised HTTPError>]


hello, get_embedding
hello, get_embedding


...

hello, insert_to_db


[2025-01-07 03:11:14,397][ERROR] Error processing site https://nogood.io/case-study/iovera: RetryError[<Future at 0x11ac14770 state=finished raised HTTPError>]
[2025-01-07 03:11:14,422][ERROR] Error processing site https://nogood.io/case-study/intuit: RetryError[<Future at 0x11a9ffe90 state=finished raised HTTPError>]


hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db


[2025-01-07 03:11:14,783][ERROR] Error processing site https://nogood.io/author/kenny-mitchell: RetryError[<Future at 0x11ac0cc50 state=finished raised HTTPError>]
[2025-01-07 03:11:14,816][ERROR] Error processing site https://nogood.io/author/yesenia-hernandez: RetryError[<Future at 0x11aa9c740 state=finished raised HTTPError>]
[2025-01-07 03:11:14,816][ERROR] Error processing site https://nogood.io/author/alexandra-liodakis: RetryError[<Future at 0x11aa9cd70 state=finished raised HTTPError>]
[2025-01-07 03:11:14,835][ERROR] Error processing site https://nogood.io/author/malika-burieva: RetryError[<Future at 0x11aa9cd10 state=finished raised HTTPError>]


KeyError occurred for site https://nogood.io/author/gayana-sarkisova: Missing description
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, insert_to_db
hello, insert_to_db
end, scrape_url


opening line: Welcome to NoGood! We're not your typical marketing agency. How can we help your brand become category-defining today?


company details: NoGood, a growth-focused marketing agency that partners with startups, scale-ups, and Fortune 500 brands to drive category-defining success.
Successfully created agent for https://nogood.io/
agent_id e86eb8cf-bc82-4985-8c27-b9b1563b57ed


In [1]:
""" CHAT WITH AGENT"""
from services.chat.lk_chat import lk_chat_process

async def test_chat(message: str , agent_id: str):
    room_name = "test_room_45iii22ij6ii"

    try:
        # Get the response generator
        response_generator = lk_chat_process(message, agent_id, room_name)
        
        print("\n\nAgent response:\n\n")
        # Stream the responses
        async for chunk in response_generator:
            # In a real application, you might want to send this to a websocket or other stream
            print('\033[96m' + chunk + '\033[0m', end='', flush=True)   

    except Exception as e:
        print(f"Error occurred: {str(e)}")

await test_chat(
    message="payment message standards to be observed by all PSPs", 
    agent_id="a706a185-fe32-4331-b35a-d241d29e9154")



Agent response:


lk_chat_process called with message: payment message standards to be observed by all PSPs, agent_id: a706a185-fe32-4331-b35a-d241d29e9154, room_name: test_room_45iii22ij6ii
response_id in lk_chat_process : 6cb78edb-c12c-49a8-95f8-1432a2e58a55
Initializing new chat history for agent_id: a706a185-fe32-4331-b35a-d241d29e9154
Initializing new chat history for room_name: test_room_45iii22ij6ii



 agents_dict: [{"id": 2, "title": "Wolfsberg Group Payment Transparency Standards 2023.pdf", "data_type": "pdf"}]
agent_metadata: RAG bot
Registered Q&A function
{"type": "response_id", "response_id": "6cb78edb-c12c-49a8-95f8-1432a2e58a55"}Created new response entry with ID: 6cb78edb-c12c-49a8-95f8-1432a2e58a55


question_and_answer func triggered with question: payment message standards to be observed by all PSPs



 agents_dict: [{"id": 2, "title": "Wolfsberg Group Payment Transparency Standards 2023.pdf", "data_type": "pdf"}]
data_source from get_agent_metadata: [{"id": 2, "t

In [12]:
from services.chat.chat import similarity_search

response = await similarity_search(
    query="tell me about the company", 
    data_source={'web': ['https://nogood.io'], 'text_files': []},
    user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

response



similarity_search...


 data_source in similarity_search: {'web': ['https://nogood.io'], 'text_files': []}
Length of results: 0


[]

### Recall Meeting bots

In [ ]:
import requests

# API endpoint
url = "https://us-west-2.recall.ai/api/v1/bot/"

# Your authorization token
token = "209a894215526cd61d70ed2f77df6cb39f6b0e4d"

# Request headers
headers = {
    "Authorization": f"Token {token}",
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "meeting_url": "https://meet.google.com/stn-gobs-xcr",
    "bot_name": "Bot"
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check response
print(response.status_code)
print(response.json())

In [ ]:
bot_joining = response.json()

bot_joining

In [ ]:
import requests

url = "https://us-west-2.recall.ai/api/v1/bot/e09e55e2-a067-473d-80be-18b9725ba7b8/"

headers = {
    "accept": "application/json",
    "Authorization": "209a894215526cd61d70ed2f77df6cb39f6b0e4d"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
response.json()